first custom model loaaded with allenai/multinews_dense_oracle dataset -  output

In [2]:
from transformers import TFT5ForConditionalGeneration
import tensorflow as tf
from transformers import TFT5ForConditionalGeneration, T5Tokenizer

In [4]:
# Specify the path to your model and tokenizer files
model_path = "/Users/andrewasher/Education/Research Project/Andrew/custom-t5-model/model"
tokenizer_path = "/Users/andrewasher/Education/Research Project/Andrew/custom-t5-model/tokenizer_custom"



In [5]:

# Load the model and tokenizer
model = TFT5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /Users/andrewasher/Education/Research Project/Andrew/custom-t5-model/model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# Define the input text that you want to summarize
input_text = """
In a charming little village, there lived a curious boy named Andy. He was known for his love of adventure and his fondness for kites. One sunny morning, Andy decided to try out a special kite his grandfather had given him – a kite with a shimmering tail. As he flew the kite in the meadow, a gust of wind pulled it higher and higher. To Andy's amazement, the kite suddenly came to life! It introduced itself as Spark and explained that it had magical powers. Spark invited Andy to hop onto the kite's tail, promising to take him on a grand adventure. Without hesitation, Andy climbed aboard. The kite soared high into the sky, and in the blink of an eye, they were floating among the clouds. They visited a candy floss cloud, a chocolate river, and a marshmallow mountain. They even met a friendly cloud creature named Fluffy, who shared stories of the sky. Andy was in awe of the magical world above. As the sun began to set, Spark gently carried Andy back to the meadow. Andy thanked Spark for the incredible adventure and watched it float back to his hand. From that day forward, Andy's kite became his loyal friend and companion, and together they embarked on many more magical adventures, exploring the wonders of the world and beyond.
"""



In [7]:
# Tokenize and generate the summary
input_ids = tokenizer.encode("summarize: " + input_text, return_tensors="tf", max_length=512, truncation=True)
summary_ids = model.generate(input_ids, max_length=100, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)


2023-10-28 11:30:49.025124: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x295daceb0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-10-28 11:30:49.025136: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-10-28 11:30:49.053246: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-28 11:30:49.146648: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-10-28 11:30:49.147454: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


In [27]:
# Decode the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print("Generated Summary:")
print(summary)

Generated Summary:
– In a charming little village in California, a boy named Andy flew his kite into a meadow with a gust of wind pulled it higher and higher. To Andy's amazement, the kite suddenly came to life! It introduced itself as Spark and explained that it had magical powers. Spark invited Andy to hop onto the kite's tail, promising to take him on a grand adventure. The sky was a candy floss cloud,


Going to fine tune with a some stories few dataset 

In [36]:
import pandas as pd

data_path = "dataset.csv"
df = pd.read_csv(data_path)


In [37]:
from transformers import T5Tokenizer

tokenizer_path = '/Users/andrewasher/Education/Research Project/Andrew/custom-t5-model/tokenizer_custom'  # Change this path to where you've saved your tokenizer.
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

MAX_TOKENS = 512
tokenized_inputs = tokenizer(df['Story'].tolist(), max_length=MAX_TOKENS, truncation=True, padding='max_length', return_tensors='tf')
tokenized_outputs = tokenizer(df['Summary'].tolist(), max_length=MAX_TOKENS, truncation=True, padding='max_length', return_tensors='tf')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [38]:
# Decode a few tokenized sequences back to text
sample_idx = [0, 1, 2]  # You can pick any indices to inspect.

print("Original vs. Decoded (Stories):")
for idx in sample_idx:
    decoded_text = tokenizer.decode(tokenized_inputs['input_ids'][idx], skip_special_tokens=True)
    print("Original:", df['Story'].iloc[idx])
    print("Decoded:", decoded_text)
    print("=" * 50)

print("\nOriginal vs. Decoded (Summaries):")
for idx in sample_idx:
    decoded_text = tokenizer.decode(tokenized_outputs['input_ids'][idx], skip_special_tokens=True)
    print("Original:", df['Summary'].iloc[idx])
    print("Decoded:", decoded_text)
    print("=" * 50)

# Print out the lengths of some tokenized sequences
print("\nTokenized Sequence Lengths (Stories):")
for idx in sample_idx:
    print(f"Length for Story {idx + 1}:", len(tokenized_inputs['input_ids'][idx]))

print("\nTokenized Sequence Lengths (Summaries):")
for idx in sample_idx:
    print(f"Length for Summary {idx + 1}:", len(tokenized_outputs['input_ids'][idx]))

# Check the shape of the tokenized tensors
print("\nShapes:")
print("Tokenized Inputs Shape:", tokenized_inputs['input_ids'].shape)
print("Tokenized Outputs Shape:", tokenized_outputs['input_ids'].shape)


Original vs. Decoded (Stories):
Original: Once upon a time in a quaint little village, there lived a kind-hearted boy named Oliver. He was known for his helpful nature and was loved by all in the village. One sunny day, as he was taking a walk in the forest, he stumbled upon an old, rusted key. Intrigued, Oliver decided to find the lock that the key belonged to. He spent days exploring the village, asking everyone if they had lost a key, but no one claimed it. Determined to solve the mystery, he ventured deep into the forest. After hours of searching, he discovered an ancient, ornate door hidden behind the thick foliage. The key fit perfectly, and with a turn, the door creaked open. To his amazement, he found a beautiful garden inside, filled with flowers of every color and trees laden with fruit. The garden, he soon learned, had been locked away for generations, forgotten by time. Oliver shared the secret with the villagers, and the garden became a place of joy and abundance for all. 

In [40]:
from transformers import TFT5ForConditionalGeneration

model_path = '/Users/andrewasher/Education/Research Project/Andrew/custom-t5-model/model'  .
model = TFT5ForConditionalGeneration.from_pretrained(model_path)


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /Users/andrewasher/Education/Research Project/Andrew/custom-t5-model/model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [42]:
import tensorflow as tf

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

def accuracy(y_true, y_pred):
    y_pred = tf.reshape(y_pred, (tf.shape(y_pred)[0], tf.shape(y_pred)[1], tf.shape(y_pred)[2]))
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.cast(y_true, tf.int32), tf.argmax(y_pred, axis=-1, output_type=tf.int32)), dtype=tf.float32))
    return accuracy

model.compile(optimizer='adam', loss=loss_function, metrics=[accuracy])

# Prepare the data for training
inputs = {
    'input_ids': tokenized_inputs['input_ids'],
    'attention_mask': tokenized_inputs['attention_mask'],
    'labels': tokenized_outputs['input_ids'],
}
pad_token_id = tokenizer.pad_token_id
inputs['decoder_input_ids'] = tf.pad(inputs['labels'][:, :-1], [[0, 0], [1, 0]], constant_values=pad_token_id)

# Training the model further on your dataset
history = model.fit(inputs, tokenized_outputs['input_ids'], batch_size=8, epochs=5, validation_split=0.1)


Epoch 1/5
3/3 [==============================] - 21s 5s/step - loss: 0.5993 - accuracy: 0.9057 - val_loss: 0.5315 - val_accuracy: 0.9092
Epoch 2/5
3/3 [==============================] - 13s 4s/step - loss: 0.4025 - accuracy: 0.9339 - val_loss: 0.4652 - val_accuracy: 0.9062
Epoch 3/5
3/3 [==============================] - 14s 4s/step - loss: 0.2883 - accuracy: 0.9440 - val_loss: 0.4524 - val_accuracy: 0.9043
Epoch 4/5
3/3 [==============================] - 14s 4s/step - loss: 0.2425 - accuracy: 0.9474 - val_loss: 0.4553 - val_accuracy: 0.9033
Epoch 5/5
3/3 [==============================] - 14s 4s/step - loss: 0.2009 - accuracy: 0.9590 - val_loss: 0.4482 - val_accuracy: 0.9072


In [43]:
save_path = '//Users/andrewasher/Education/Research Project/Andrew/finetuned model/'
model.save_pretrained(save_path + 'model')
tokenizer.save_pretrained(save_path + 'tokenizer_custom')


('//Users/andrewasher/Education/Research Project/Andrew/finetuned model/tokenizer_custom/tokenizer_config.json',
 '//Users/andrewasher/Education/Research Project/Andrew/finetuned model/tokenizer_custom/special_tokens_map.json',
 '//Users/andrewasher/Education/Research Project/Andrew/finetuned model/tokenizer_custom/spiece.model',
 '//Users/andrewasher/Education/Research Project/Andrew/finetuned model/tokenizer_custom/added_tokens.json')

okay now output

In [1]:
from transformers import TFT5ForConditionalGeneration


In [2]:
import tensorflow as tf
from transformers import TFT5ForConditionalGeneration, T5Tokenizer


In [3]:
# Specify the path to your model and tokenizer files
model_path = "/Users/andrewasher/Education/Research Project/Andrew/finetuned model/model"
tokenizer_path = "/Users/andrewasher/Education/Research Project/Andrew/finetuned model/tokenizer_custom"



In [4]:

# Load the model and tokenizer
model = TFT5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /Users/andrewasher/Education/Research Project/Andrew/finetuned model/model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# Define the input text that you want to summarize
input_text = """In a small, quiet village, there lived a poor artist named Maya. She longed for a magical paintbrush that could bring her paintings to life. One day, as she sat by the river, a talking frog appeared. The frog granted her wish, giving her a paintbrush that could make her dreams a reality. Maya’s first creation was a magnificent tree, and as soon as she painted it, it sprouted in her garden. Excitement filled her heart as she realized the power of the brush. She painted a river that flowed with crystal-clear water and a sky filled with brilliant stars. But soon, greed took over her heart. She painted a grand palace and began to live a life of luxury. Her friends and family, concerned by her change, tried to remind her of the happiness she once had. One day, Maya painted a beautiful bird and wished for it to sing. Instead, it let out a mournful cry. She realized that her magical brush had lost its power because she had used it for selfish desires. Filled with regret, Maya painted a simple, humble cottage and wished for her old life back. The cottage appeared, and her heart felt light once more. She returned to her village, where she shared her story and the lessons she had learned about the true meaning of happiness and contentment. The magical paintbrush remained in her hands, but she vowed to use it only for good, creating art that would inspire and bring joy to others. These short stories offer a glimpse into different worlds and lessons about kindness, curiosity, and the consequences of our choices.
"""

In [6]:
# Tokenize and generate the summary
input_ids = tokenizer.encode("summarize: " + input_text, return_tensors="tf", max_length=512, truncation=True)
summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)


2023-10-30 23:15:56.335282: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x11838c880 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-10-30 23:15:56.335293: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-10-30 23:15:56.368334: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-30 23:15:56.505306: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-10-30 23:15:56.506831: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


In [7]:
# Decode the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print("Generated Summary:")
print(summary)

Generated Summary:
a poor artist in a small village has lived a poor artist named Maya. She longed for a magical paintbrush that could bring her dreams to life. One day, Maya, a talking frog granted her wish, giving her a magical brush, a magical tree, and as soon as she painted it, it sprouted in her garden. Excitement filled her heart. She realized that her magical brush lost its power, painted a simple cottage and wished for her old life back. She returned to her village, sharing her story and lessons about kindness, curiosity, and the consequences of our choices.
